## Variables to set

In [1]:
# set the model type
estimModel = 'nbinom' #nbinom or hurdle

# list of the (prediction) windows
max_w = 24
window_list = list(range(2, max_w+1))

### Import packages and define functions

In [2]:
import pandas as pd
import numpy as np
import os
import CRPS.CRPS as pscore
import copy
from joblib import dump, load
from scipy.stats import nbinom, poisson
from time import sleep
from tqdm import tqdm
import warnings

## functions for the distribtion models
# cdf of the truncated negative binomial distribution
def truncNbinomCdf(y, n, p, log=True):

    ## error/input handling part one
    # n and p have to be greater than zero and p <= 1
    if n <= 0 or p <= 0 or p > 1:
        if not isinstance(y, (list, np.ndarray)):
            return np.nan
        else:
            return np.full(len(y), np.nan)

    ## calculation
    is_scalar = np.isscalar(y)  # check if y is scalar or array

    if is_scalar:
        y = np.array([y])  # typecast scalar to onedimensional array
    elif isinstance(y, list):
        y = np.array(y)

    f_zero = nbinom.pmf(0, n, p) # f(0) untruncated density

    # general formula for lower trunc. distributions
    cdf_y = (nbinom.cdf(y, n, p) - nbinom.cdf(0, n, p)) / (1 - f_zero)
    ## error/input handling part two
    # VERY IMPORTANT STEP: there might be numerical instabilities, which may lead to
    # nbinomo.cdf(x) < nbinom.cdf(0)!!!!! this then leads to negative values of the cdf
    # or nans in the log version (np.log(neg number))
    cdf_y[cdf_y < 0] = 0
    # set values to 0, if y <= 0 (y <=0 not allowed per definition of a 0 truncated count distribution)
    cdf_y[y <= 0] = 0

    # in case of log CDF
    if log: 
        # ignore the 'RuntimeWarning: divide by zero encountered in log' warning (np.log(0))
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        log_cdf_y = np.log(cdf_y) # general formula for lower trunc. distributions
        warnings.filterwarnings('default', category=RuntimeWarning)

        if is_scalar:
            return log_cdf_y[0]  # return scalar, if onedimensional array
        else:
            return log_cdf_y
    # normal CDF
    else:
        if is_scalar:
            return cdf_y[0]  
        else:
            return cdf_y
    

#log.p	logical; if TRUE, probabilities p are given as log(p)    
def qnbinom_trunc(p, nNbinom, pNbinom, log_p=False):
    ## calculation of the quantile
    # if f(0)=0 no truncation is needed
    if nbinom.pmf(0, nNbinom, pNbinom) == 0:
        return nbinom.ppf(p, nNbinom, pNbinom) # if p=0, -1 is returned instead of 0 (0 is truncated)
        # but this is not that important and is ignored here (because the 0 quantile does not make sense)
    else:
        # n and p have to be greater than zero and p <= 1
        if nNbinom <= 0 or pNbinom <= 0 or pNbinom > 1:
            if not isinstance(p, (list, np.ndarray)):
                return np.nan
            else:
                return np.full(len(p), np.nan)

        # Convert p (quantile) to array if it's a scalar
        if not isinstance(p, (list, np.ndarray)):
            p = np.array([p])
        elif isinstance(p, list):
            p = np.array(p)
        
        n = len(p) # number of quantiles

        # Set log-probabilities (lower tail)
        if log_p:
            logp = p
        else:
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            logp = np.log(p)
            warnings.filterwarnings('default', category=RuntimeWarning)
        
        # error handling/deal with special cases (outputs NA and Inf)
        quantiles = np.full(n, np.nan)
        na = np.isnan(logp) # nan <-> p < 0 -> return nan
        neginf = np.isneginf(logp) # -inf <-> p = 0 -> return 0 (due to truncation, otherwise -1)
        zero = logp == 0  # 0 <-> p = 1 -> return inf
        aboveZero = logp > 0 # >0 <-> p > 1 -> return nan

        # set quantile array if one of the restrictions is not fulfilled
        quantiles[neginf] = 0
        quantiles[zero] = np.inf
        quantiles[aboveZero] = np.nan
        
        # mask array, true if conditions are fulfilled
        mask = np.logical_not(np.logical_or(na, np.logical_or(neginf, np.logical_or(zero, aboveZero))))
        validLogp = logp[mask]

        if len(validLogp) == 0:
            # Return output
            if len(quantiles) == 1:
                return quantiles[0] # if single quantile is handed over
            else:
                return quantiles

        # find valid max value with mask
        lp_max = np.max(validLogp)
        p_max = np.exp(lp_max)

        # calculate mean and variance out of n and p
        mean = (nNbinom * (1 - pNbinom)) / pNbinom
        var = (nNbinom * (1 - pNbinom)) / (pNbinom**2)

        # find an adequate upper limit, starting from the extreme conservative chebychev inequality
        upper = int(mean + np.sqrt(var/(1-np.exp(lp_max)))) #Chebychev inequality

        # if upper < 1000 there is an log(0)=-inf with warning -> ignore this warning
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        # lower the upper limit (saves computation time)
        while truncNbinomCdf(upper-1000, nNbinom, pNbinom, log=False) > p_max:
            upper = upper - 1000

        # after this section warnings are enabled again
        warnings.filterwarnings('default', category=RuntimeWarning)

        yarray = np.arange(1, int(upper)+1) # the y values for which the CDF is going to be calculated
        logcdf = truncNbinomCdf(yarray, nNbinom, pNbinom) # calculate log CDF (faster computation time)

        # Compute output
        for i in range(n): # for all quantiles   
            if not na[i] and not neginf[i] and not zero[i] and not aboveZero[i]:
                    quantiles[i] = np.sum(logcdf < np.array(logp[i])) + 1 #+1 because 0 is truncated
        
        # Return output
        if len(quantiles) == 1:
            return quantiles[0] # if single quantile is handed over
        else:
            return quantiles

# truncated poisson---------------------------------------------------
# cdf of the truncated poisson distribution
def truncPoisCdf(y, mu, log=True):
    # values of lam <= 0 (0 truncation -> 0= not allowed) are not allowed. return nan
    if mu <= 0:
        if not isinstance(y, (list, np.ndarray)):
            return np.nan
        else:
            return np.full(len(y), np.nan)

    is_scalar = np.isscalar(y)  # check if y is scalar or array

    if is_scalar:
        y = np.array([y])  # typecast scalar to onedimensional array
    elif isinstance(y, list):
        y = np.array(y)

    f_zero = poisson.pmf(0, mu) # f(0) untruncated density

    # general formula for lower trunc. distributions
    cdf_y = (poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero)
    # VERY IMPORTANT STEP: there might be numerical instabilities, which may lead to
    # poisson.cdf(x) < poisson.cdf(0)!!!!! this then leads to negative values of the cdf
    # or nans in the log version (np.log(neg number))
    cdf_y[cdf_y < 0] = 0 # set negative values to zero
    # set values to 0, if y <= 0 (y <=0 not allowed per definition of a 0 truncated count distribution)
    cdf_y[y <= 0] = 0

    # in case of log CDF
    if log:
        # ignore the 'RuntimeWarning: divide by zero encountered in log' warning (np.log(0))
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        log_cdf_y = np.log(cdf_y) # general formula for lower trunc. distributions
        warnings.filterwarnings('default', category=RuntimeWarning)

        if is_scalar:
            return log_cdf_y[0]  # return scalar, if onedimensional array
        else:
            return log_cdf_y
    # normal CDF    
    else:
        if is_scalar:
            return cdf_y[0]
        else:
            return cdf_y
        
def log1mexp(x):
    if np.any((x < 0) & (~np.isnan(x))):
        raise ValueError("Inputs need to be non-negative!")
    return np.where(x <= np.log(2), np.log(-np.expm1(-x)), np.log1p(-np.exp(-x)))
        

#log.p	logical; if TRUE, probabilities p are given as log(p)    
def qpois_trunc(p, lam, log_p=False):
    ## calculation of the quantile
    # if f(0)=0 no truncation is needed
    if poisson.pmf(0, lam) == 0:
        return poisson.ppf(p, lam)
    else:
        # values of lam <= 0 (0 truncation -> 0= not allowed) are not allowed. return nan
        if lam <= 0:
            if not isinstance(p, (list, np.ndarray)):
                return np.nan
            else:
                return np.full(len(p), np.nan)


        # Convert p (quantile) to array if it's a scalar
        if not isinstance(p, (list, np.ndarray)):
            p = np.array([p])
        elif isinstance(p, list):
            p = np.array(p)
        
        n = len(p) # number of quantiles

        # Set log-probabilities (lower tail)
        if log_p:
            logp = p
        else:
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            logp = np.log(p)
            warnings.filterwarnings('default', category=RuntimeWarning)
        
        # error handling/deal with special cases (outputs NA and Inf)
        quantiles = np.full(n, np.nan)
        na = np.isnan(logp) # nan <-> p < 0 -> return nan
        neginf = np.isneginf(logp) # -inf <-> p = 0 -> return 0 (due to truncation, otherwise -1)
        zero = logp == 0  # 0 <-> p = 1 -> return inf
        aboveZero = logp > 0 # >0 <-> p > 1 -> return nan

        # set quantile array if one of the restrictions is not fulfilled
        quantiles[neginf] = 0
        quantiles[zero] = np.inf
        quantiles[aboveZero] = np.nan
        
        # mask array, true if conditions are fulfilled
        mask = np.logical_not(np.logical_or(na, np.logical_or(neginf, np.logical_or(zero, aboveZero))))
        validLogp = logp[mask]

        if len(validLogp) == 0:
            # Return output
            if len(quantiles) == 1:
                return quantiles[0] # if single quantile is handed over
            else:
                return quantiles

        # find valid max value with mask
        lp_max = np.max(validLogp)
        p_max = np.exp(lp_max)

        # find an adequate upper limit, starting from the extreme conservative chebychev inequality
        upper = int(lam + np.sqrt(lam * np.exp(-log1mexp(-lp_max)))) #Chebychev inequality

        # if upper < 1000 there is an log(0)=-inf with warning -> ignore this warning
        warnings.filterwarnings('ignore', category=RuntimeWarning)

        while truncPoisCdf(upper-1000, lam, log=False) > p_max:
            upper = upper - 1000

        # after this section warnings are enabled again
        warnings.filterwarnings('default', category=RuntimeWarning)

        yarray = np.arange(1, int(upper)+1) # the y values for which the CDF is going to be calculated
        logcdf = truncPoisCdf(yarray, lam) # calculate log CDF (faster computation time)

        # Compute output
        for i in range(n): # for all quantiles   
            if not na[i] and not neginf[i] and not zero[i] and not aboveZero[i]:
                    quantiles[i] = np.sum(logcdf < np.array(logp[i])) + 1 #+1 because 0 is truncated
        
        # Return output
        if len(quantiles) == 1:
            return quantiles[0] # if single quantile is handed over
        else:
            return quantiles

### function to compute distribution-------------------------------------------------
def baseFatalModel_quantiles(featureSeries, quantiles, w=None, model='hurdle'):
    # list to store quantiles 
    dummy_fatalities_list = []
    # string to store model distribution
    dist_string = ''

    mean = None
    var = None

    numberQuantiles = len(quantiles)

    # hurdle model
    if model == 'hurdle':
        if w == None:
            
            # calculate pt, i.e. the probability that y>0
            p_t = 1 - (featureSeries.value_counts().get(0, 0) / featureSeries.count())
            # calculate mean and variance of the features
            mean = pd.Series.mean(featureSeries[featureSeries != 0])
            var = pd.Series.var(featureSeries[featureSeries != 0])

        elif w <= 0:
            raise ValueError("w has to be greater than zero")
        
        else:
            features = featureSeries.tail(w).loc[:,'ged_sb']
            # calculate pt, i.e. the probability that y>0
            p_t = 1 - (features.value_counts().get(0, 0) / features.count())
            # calculate mean and variance of the features
            mean = pd.Series.mean(features[features != 0])
            var = pd.Series.var(features[features != 0])

        # pd.Series.var or mean returns Nan in case of a passed series of length 1
        if np.isnan(mean):
            mean = 0
        if np.isnan(var):
            var = 0

        # check if there are values above zero, otherwise no second component (trunc dist.) needed
        if p_t > 0:
            # component 1, y=0: Bernoulli
            comp2_quantiles = [q for q in quantiles if q > (1-p_t)] #quantiles for the second component
            removed_elements_length = numberQuantiles-len(comp2_quantiles)
            zeros_array = np.zeros(removed_elements_length) #zero values that originate from the bernoulli dist

            # component 2, y>0
            if var != 0 and var > mean:
                # calculate n (r) and p via average/variance
                n = (mean**2) / (var - mean) # equivalent to r
                p = mean / var

                trunc_nbinom_quantiles = qnbinom_trunc(comp2_quantiles, n, p)

                dummy_fatalities_list = np.concatenate((zeros_array, trunc_nbinom_quantiles)).tolist()
                dist_string = 'BernoullitruncNbinom'

            else:  # equivalent to all means and 0 < var <= mean (mean cant be 0, because of hurdle)
                trunc_pois_quantiles = qpois_trunc(comp2_quantiles, mean)
                
                dummy_fatalities_list = np.concatenate((zeros_array, trunc_pois_quantiles)).tolist()
                dist_string = 'BernoulliTruncPois'
            
        # p_t = 0 so no second component is needed    
        else:
            dummy_fatalities_list = [0] * numberQuantiles
            dist_string = 'BernoulliHurdle'

    # nbinom model
    elif model == 'nbinom':
        if w == None:
             # calculate mean and variance of the features
            mean = pd.Series.mean(featureSeries)
            var = pd.Series.var(featureSeries)
        elif w <= 0:
            raise ValueError("w has to be greater than zero")
        else:
            # calculate mean and variance of the features
            mean = pd.Series.mean(featureSeries.tail(w).loc[:,'ged_sb'])
            var = pd.Series.var(featureSeries.tail(w).loc[:,'ged_sb'])

        if var != 0 and var > mean:
                # calculate n (r) and p via average/variance
                n = (mean**2) / (var - mean) # equivalent to r
                p = mean / var

                dummy_fatalities_list = nbinom.ppf(quantiles, n, p).tolist()
                dist_string = 'NBinom'

        elif mean == 0 and var == 0: # due to faster calculation
                dummy_fatalities_list = [0] * numberQuantiles
                dist_string = 'Pois'

        else: # equivalent to all means and 0 < var <= mean
                dummy_fatalities_list = poisson.ppf(quantiles, mean).tolist()
                dist_string = 'Pois'

    return {'fatalities': dummy_fatalities_list, 'dist': dist_string, 'mean': mean, 'var': var}

In [3]:
# last feature year
feature_year = '2024'

prediction_year = '2024/25'

# path to the current directory
current_dir = os.getcwd()

# read feature dataset
relative_path_features = os.path.join('..','..', 'data', 'cm_features.parquet')
path_features = os.path.join(current_dir, relative_path_features)

feature_data_toApr2024 = pd.read_parquet(path_features, engine='pyarrow')
feature_data_toApr2024.set_index(['month_id', 'country_id'], inplace=True)

country_list = sorted(feature_data_toApr2024.index.get_level_values('country_id').unique().tolist())

# country group list of all years
country_feature_group_list = []

# fill list 
country_feature_group_list.append(feature_data_toApr2024.groupby('country_id'))

### List of the countries for which a prediction is requested

In [4]:
relative_path_countrylist = os.path.join('..', '..', 'data', 'country_list.csv')
path_countrylist = os.path.join(current_dir, relative_path_countrylist)

# CSV-Datei einlesen und als Pandas-Datensatz speichern
countryList_prediction = pd.read_csv(path_countrylist)
country_list_views = countryList_prediction.loc[:,'country_id'].values.tolist() 

month_list = []
countries_to_remove = []
for country_id in country_list:

    if country_id in country_list_views:
        feature_data = country_feature_group_list[0].get_group(country_id)

        # numbers of months from the feature dataset
        month_list_feature_data_original = feature_data.index.get_level_values('month_id').tolist()
        number_months_feature_data = len(month_list_feature_data_original) 
        
    else:
        countries_to_remove.append(country_id)

country_list = list(set(country_list) - set(countries_to_remove))
month_list.sort()
len(country_list)

191

## Baseline 1-4

Optimize **w** (through the CRPS) regarding
|            | datasets    | countries   | prediction windows |
|------------|-------------|-------------|--------------------|
| baseline 1 | all         | all         | all                |
| baseline 2 | all         | inidvidual  | all                |
| baseline 3 | all         | all         | individual         |
| baseline 4 | all         | inidvidual  | individual         |

### The minimization is based on calculating the quantiles for each country, w and year (of the four datasets).
#### List of representative countries

In [5]:
#subsample_list = [1, 57, 235, 13, 121, 223, 220]
#country_list = [x for x in country_list if x in subsample_list]

In [6]:
# modify country_list so that it contains only country_ids 
# that have at least the last n months of observations in the last dataset (2020)!
numberMonths_toApr24 = 138 # 120 = 5*12 (5 jahre für 2017) + 5*12 (jedes Jahr 12 Monate mehr also 2020 10 Jahre) + 12 (23) + 6 (Oktober bis April)
#ABER: um konsistent zu bleiben wird für jedes Jahr (jeden to_octX Datensatz) nur die letzten 5 Jahre verwendet!!!


s_prediction_list = list(range(3, 15))

number_countries = len(country_list)
number_w = len(window_list)

# lists for the estimation of the distribution
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies
string_quantile_list = [f"{round(q * 100, 1)}%" for q in quantiles] # sting values of the quantiles

# last month of the dataframe as reference for the moving prediction windows
last_month = feature_data_toApr2024.index.get_level_values('month_id').tolist()[-1]

# list to store the estimations/predictions for each w
baseline_estimate_list = []

# loop through windows
for i in tqdm(range(number_w)):

    w = window_list[i] # current window
    baseline_estimate_list.append({'window':w, 
                                'country_predict_list':[{'country_id':country, 'predictionWindowsN':[]} for country in country_list]})
    
    #calculate the number of subsets, that are used to estimate the distribution and validate it via 12 months of actuals 
    # the number is dependent of the actual w. E.g. with the maximal w (e.g. 24): if w=24, actuals are 12 months (starting with s=3 to s=14) 
    # -> 24 + 2 + 12 = 39 observations of ged_sb per window
    # so if the dataset has 120 observations there are 120 - 38 = 82 shiftable windows for 2020
    numberWindows = numberMonths_toApr24 - (w + 2 + 12)

    windowLength = w + 2 + 12 # length of the individual window for the current w
    
    # loop through all countries
    for index in range(number_countries):

        country = country_list[index]

        features = country_feature_group_list[-1].get_group(country) # features of country
        

        # loop through all X equal parts of the feature dataset (traindata length w, actuals is vector of the next t+3 till t+12 observations)
        for j in range(numberWindows):
            starting_month_window = last_month - windowLength + 1 - numberWindows + 1  + j
            ending_month_window = starting_month_window + w - 1

            starting_month_actuals = ending_month_window + 3
            ending_month_actuals = starting_month_actuals + 11
            
            window_features = features.loc[(slice(starting_month_window, ending_month_window), slice(None)), 'ged_sb']
            window_actuals = features.loc[(slice(starting_month_actuals, ending_month_actuals), slice(None)), 'ged_sb']

            #predict = nBinom_quantiles(window_features, 'None', quantiles)
            predict = baseFatalModel_quantiles(window_features, quantiles, model=estimModel)
            
            baseline_estimate_list[i]['country_predict_list'][index]['predictionWindowsN'].append(
                [{'country_id':country, 'w':w, 'dist':predict['dist'], 
                'mean':predict['mean'], 'var':predict['var'], 'first_month_feature':starting_month_window, 
                'quantile':string_quantile_list, 'fatalities':predict['fatalities']}, 
                {'s':s_prediction_list, 
                    'month_id': window_actuals.index.get_level_values('month_id'),
                    'unreal_actuals':window_actuals.values}])

100%|██████████| 23/23 [01:31<00:00,  3.98s/it]


In [7]:
#                     w                        country         rollingWindow|predicton/actuals
#baseline_estimate_list[0]['country_predict_list'][177]['predictionWindowsN'][8][0]

#### Compute the average CRPS over all indiviual moving windows per w and year

In [8]:
# lists to store all crps values
baseline_crps_list_toApr2024 = [
    {
        'country_id': country,
        'baseline': [
            {'s': s, 'w': [], 'CRPS': []}
            for s in s_prediction_list
        ]
    }
    for country in country_list
]
baseline_crps_list_toApr2024 = copy.deepcopy(baseline_crps_list_toApr2024)

# number of prediction windows
number_s = len(s_prediction_list)

# fill lists with crps calculations
for s in tqdm(s_prediction_list):

    for index in range(number_countries):
        country = country_list[index]
            
        for i in range(number_w):
            w = window_list[i]
            dummy_crps_list = [] 

            # loop over all subset windows of the country and w 
            for j in range(len(baseline_estimate_list[i]['country_predict_list'][index]['predictionWindowsN'])):

                distribution = baseline_estimate_list[i]['country_predict_list'][index]['predictionWindowsN'][j][0]['fatalities']
                actual = baseline_estimate_list[i]['country_predict_list'][index]['predictionWindowsN'][j][1]['unreal_actuals'][s-3]

                crps = pscore(np.array(distribution),actual).compute()[0]

                dummy_crps_list.append(crps)

            
            # dataframe toApr2024
            baseline_crps_list_toApr2024[index]['baseline'][s-3]['w'].append(w)
            baseline_crps_list_toApr2024[index]['baseline'][s-3]['CRPS'].append(np.mean(dummy_crps_list))

task2_baseline_list = [baseline_crps_list_toApr2024]


100%|██████████| 12/12 [16:41<00:00, 83.46s/it]


#### Minimization
'w_minimization_list' contains the minimal w's for the different baselines for each year

In [9]:
# list to store the results of the minimal w's
w_minimization_list = [{'predictionYear':prediction_year, 'minWData':[]}]

# list to store the list to compute the minimal w's
w_compute_list = [{'predictionYear':prediction_year, 'data':[]}]

# loop over the four different datasets to predict (18-21)

v1_baseline_crps_dict = {'w':[],'CRPS':[]}
v2_baseline_crps_list = [{'country_id': country, 'baseline': {'w':[],'CRPS':[]}} for country in country_list]
v3_baseline_crps_list = [{'s':s,'w':[],'CRPS':[]} for s in s_prediction_list]

## baseline v1---------------------------------------------------------------------------
# loop over w
for j in range(number_w):
    w = window_list[j]
    dummy_crps_v1_list = []
    # loop over countries
    for i in range(number_countries):
        # loop over prediction windows s
        for k in range(number_s):
            dummy_crps_v1_list.append(task2_baseline_list[0][i]['baseline'][k]['CRPS'][j])
    v1_baseline_crps_dict['w'].append(w)
    v1_baseline_crps_dict['CRPS'].append(np.mean(dummy_crps_v1_list))

v1_baseline_crps = pd.DataFrame(v1_baseline_crps_dict)

w_compute_list[0]['data'].append(v1_baseline_crps)

v1_baseline_crps = v1_baseline_crps[v1_baseline_crps.CRPS == v1_baseline_crps.loc[:,'CRPS'].min()]
v1_baseline_crps.set_index(pd.Index(range(len(v1_baseline_crps))), inplace=True)
    
w_minimization_list[0]['minWData'].append(v1_baseline_crps)
#----------------------------------------------------------------------------------------

## baseline v2----------------------------------------------------------------------------
# list for baseline v2
for i in range(number_countries):
    for j in range(number_w):
        w = window_list[j]
        dummy_crps_v2_list = []
        for k in range(number_s):
            dummy_crps_v2_list.append(task2_baseline_list[0][i]['baseline'][k]['CRPS'][j])
        v2_baseline_crps_list[i]['baseline']['w'].append(w)
        v2_baseline_crps_list[i]['baseline']['CRPS'].append(np.mean(dummy_crps_v2_list))
    
# dataframe with the w that minimizes the CRPS for every country (v2)
data_v2 = {
    'country_id':[],
    'w':[],
    'CRPS':[]
}
for i in range(len(v2_baseline_crps_list)):
    # get the index of the minimal CRPS value
    min_index = v2_baseline_crps_list[i]['baseline']['CRPS'].index(min(v2_baseline_crps_list[i]['baseline']['CRPS']))
    
    # store values in dict
    data_v2['country_id'].append(v2_baseline_crps_list[i]['country_id'])
    data_v2['w'].append(v2_baseline_crps_list[i]['baseline']['w'][min_index])
    data_v2['CRPS'].append(v2_baseline_crps_list[i]['baseline']['CRPS'][min_index])
    
v2_baseline_crps = pd.DataFrame(data_v2)
w_minimization_list[0]['minWData'].append(v2_baseline_crps)
w_compute_list[0]['data'].append(v2_baseline_crps_list)
#----------------------------------------------------------------------------------------


## baseline v3---------------------------------------------------------------------------
for s_index in range(number_s):
    dummy_crps_v3_list = []
    s = s_prediction_list[s_index]
    for w_index in range(number_w):
        w = window_list[w_index]
        for i in range(number_countries):
            dummy_crps_v3_list.append(task2_baseline_list[0][i]['baseline'][s_index]['CRPS'][w_index])
        v3_baseline_crps_list[s_index]['w'].append(w)
        v3_baseline_crps_list[s_index]['CRPS'].append(np.mean(dummy_crps_v3_list))

# dataframe with the w that minimize the CRPS for each prediction window s
data_v3 = {
    's':[],
    'w':[],
    'CRPS':[]
}
# length of the v3_baseline list is the number of prediction windows
for i in range(len(v3_baseline_crps_list)):
    s = s_prediction_list[i]
    # get the index of the minimal CRPS value
    min_index = v3_baseline_crps_list[i]['CRPS'].index(min(v3_baseline_crps_list[i]['CRPS']))

    # store values in dict
    data_v3['s'].append(s)
    data_v3['w'].append(v3_baseline_crps_list[i]['w'][min_index])
    data_v3['CRPS'].append(v3_baseline_crps_list[i]['CRPS'][min_index])

v3_baseline_crps = pd.DataFrame(data_v3)

w_minimization_list[0]['minWData'].append(v3_baseline_crps)
w_compute_list[0]['data'].append(v3_baseline_crps_list)
#----------------------------------------------------------------------------------------

## baseline v4---------------------------------------------------------------------------
v4_baseline_crps = [{'country_id':country,
                    's':[],
                    'w':[],
                    'CRPS':[]
                    } for country in country_list]

# loop over all countries
for i in range(len(task2_baseline_list[0])):
    # loop over all prediction windows
    for s_index in range(number_s):
        s = s_prediction_list[s_index]
        # get the index of the minimal CRPS value
        min_index = task2_baseline_list[0][i]['baseline'][s_index]['CRPS'].index(min(task2_baseline_list[0][i]['baseline'][s_index]['CRPS']))
    
        # store values in dict
        v4_baseline_crps[i]['s'].append(s)
        v4_baseline_crps[i]['w'].append(task2_baseline_list[0][i]['baseline'][s_index]['w'][min_index])
        v4_baseline_crps[i]['CRPS'].append(task2_baseline_list[0][i]['baseline'][s_index]['CRPS'][min_index])

    v4_baseline_crps[i] = pd.DataFrame(v4_baseline_crps[i])

w_minimization_list[0]['minWData'].append(v4_baseline_crps)
w_compute_list[0]['data'].append(task2_baseline_list[0])
#----------------------------------------------------------------------------------------

In [10]:
#                dataset/year |baseline method|country (only baseline 4)
w_minimization_list[0]['minWData'][1]

,country_id,w,CRPS
0,1,2,0.000000
1,13,15,0.657637
2,57,9,2694.937713
3,121,2,0.000000
4,220,24,763.362668
5,223,8,11.720113
6,235,7,0.481448


## Prediction with the estimated baseline models
With the w values for the four different baselines computed above the prediction is calculated and evaluated in the following.

In [13]:
# list to save the predictions for each country
baseline_prediction_list = [[{'country_id': country, 'base': 1, 'prediction': {prediction_year: []}} for country in country_list],
                            [{'country_id': country, 'base': 2, 'prediction': {prediction_year: []}} for country in country_list],
                            [{'country_id': country, 'base': 3, 'prediction': {prediction_year: []}} for country in country_list],
                            [{'country_id': country, 'base': 4, 'prediction': {prediction_year: []}} for country in country_list]]

quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies
string_quantile_list = [f"{round(q * 100, 1)}%" for q in quantiles]


# loop through all countries (that are present in each dataset)
for index in range(number_countries):
    country = country_list[index]
    
    features = country_feature_group_list[0].get_group(country) # features of country in dataset i

    # loop over the four different baseline minimization methods
    for j in range(len(w_minimization_list[0]['minWData'])):

        # baseline 1
        if j == 0:
            w = w_minimization_list[0]['minWData'][j].loc[0,'w'] # use the w obtained by minimization on the feature dataset
            #fit = nBinom_quantiles(features, w, quantiles) # calculate the quantiles for the w
            fit = baseFatalModel_quantiles(features, quantiles, w=w, model=estimModel)

            baseline_prediction_list[j][index]['prediction'][prediction_year].append({'s':s_prediction_list, 'w':w, 
                                                                                'dist':fit['dist'], 'mean':fit['mean'],
                                                                                'var':fit['var'], 
                                                                                'quantile':string_quantile_list, 
                                                                                'fatalities':fit['fatalities']})

        # baseline 2
        elif j == 1:
            if country == w_minimization_list[0]['minWData'][j].loc[index,'country_id']:
                w = w_minimization_list[0]['minWData'][j].loc[index,'w']
                #fit = nBinom_quantiles(features, w, quantiles) # calculate the quantiles for the w
                fit = baseFatalModel_quantiles(features, quantiles, w=w, model=estimModel)

                baseline_prediction_list[j][index]['prediction'][prediction_year].append({'s':s_prediction_list, 'w':w, 
                                                                                    'dist':fit['dist'], 'mean':fit['mean'],
                                                                                    'var':fit['var'], 
                                                                                    'quantile':string_quantile_list, 
                                                                                    'fatalities':fit['fatalities']})
            else:
                print('Stopp')
                break

        # baseline 3
        elif j == 2:
            for s in s_prediction_list:
                w = w_minimization_list[0]['minWData'][j].loc[s-3,'w']
                #fit = nBinom_quantiles(features, w, quantiles) # calculate the quantiles for the w
                fit = baseFatalModel_quantiles(features, quantiles, w=w, model=estimModel)

                baseline_prediction_list[j][index]['prediction'][prediction_year].append({'s':s, 'w':w, 
                                                                                    'dist':fit['dist'], 'mean':fit['mean'],
                                                                                    'var':fit['var'], 
                                                                                    'quantile':string_quantile_list, 
                                                                                    'fatalities':fit['fatalities']})

        # baseline 4
        elif j == 3:
            if country == w_minimization_list[0]['minWData'][j][index].loc[0,'country_id']:
                for s in s_prediction_list:
                    w = w_minimization_list[0]['minWData'][j][index].loc[s-3,'w']
                    #fit = nBinom_quantiles(features, w, quantiles) # calculate the quantiles for the w
                    fit = baseFatalModel_quantiles(features, quantiles, w=w, model=estimModel)

                    baseline_prediction_list[j][index]['prediction'][prediction_year].append({'s':s, 'w':w, 
                                                                                        'dist':fit['dist'], 'mean':fit['mean'],
                                                                                        'var':fit['var'], 
                                                                                        'quantile':string_quantile_list, 
                                                                                        'fatalities':fit['fatalities']})
            else:
                print('Stopp')
                break

In [30]:
#          baseline method|country                  s-3 (only index 0 for baseline 1 and 2 due to the non minimizing s)
#baseline_prediction_list[2][2]['prediction'][prediction_year][11]['fatalities']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [ ]:
# save variables in joblib files
variable_string = str(estimModel)+'Wmax'+str(max_w)
file1name = 'FinalTask1_baseline_estim_' + variable_string + '.joblib'
file2name = 'FinalTask1_baseline_predct_' + variable_string + '.joblib'

dump([country_list, baseline_estimate_list], 
       file1name)

dump([task2_baseline_list, w_minimization_list, baseline_prediction_list], 
       file2name)

['FinalTask2_baseline_predct_hurdleWmax24.joblib']